In [0]:
from pyspark.sql import functions as F
from delta.tables import DeltaTable

In [0]:
%run /Workspace/consolidated_pipeline/databricks-lakehouse-etl-pipeline/setup/utilities

In [0]:
print(bronze_schema, silver_schema, gold_schema)